In [58]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [51]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [52]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season', 'timestamp', 'totalHarvestablePods', 'totalHarvestedPods', 'totalPods']) 

In [68]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns={
    'totalHarvestablePods': 'harvestable_pods', 
    'totalHarvestedPods': 'harvested_pods', 
    'totalPods': 'total_pods', 
})
df_field['unharvestable_pods'] = df_field.total_pods - df_field.harvestable_pods
# TODO: Quick and dirty fix bc the subgraph is broken here. 
# See subgraph issue https://github.com/BeanstalkFarms/Beanstalk-Subgraph/issues/38
df_field.loc[df_field.unharvestable_pods < 0, 'unharvestable_pods'] = 0
df_field['harvestable_pods'] = df_field.harvestable_pods - df_field.harvested_pods 
# df_field['unharvestable_ratio'] = df_field.unharvestable_pods / df_field.total_pods
df_field.head()

,season,harvestable_pods,harvested_pods,timestamp,total_pods,unharvestable_pods
0,25,5049.266341,0.000000,2021-08-07 00:06:08,4899.059312,0.0
1,49,337.098080,15526.430505,2021-08-08 00:08:15,2502.841552,0.0
2,73,1479.961469,21827.924952,2021-08-09 00:15:57,6821.395704,0.0
3,97,7611.092329,25843.746295,2021-08-10 00:01:29,0.000000,0.0
4,121,6904.487041,26550.351583,2021-08-11 00:00:05,0.000000,0.0


In [69]:
id_vars = ['timestamp', 'season']
value_vars = [
   'unharvestable_pods', 'harvested_pods', 'harvestable_pods', 
   # 'unharvestable_ratio'
]
df = wide_to_longwide(df_field, "timestamp", id_vars, value_vars, sort_col="timestamp") 
df.head()

,timestamp,season,variable,value,unharvestable_pods,harvested_pods,harvestable_pods
0,2021-08-07 00:06:08,25,unharvestable_pods,0.000000,0.0,0.000000,5049.266341
1,2021-08-07 00:06:08,25,harvested_pods,0.000000,0.0,0.000000,5049.266341
2,2021-08-07 00:06:08,25,harvestable_pods,5049.266341,0.0,0.000000,5049.266341
3,2021-08-08 00:08:15,49,unharvestable_pods,0.000000,0.0,15526.430505,337.098080
4,2021-08-08 00:08:15,49,harvested_pods,15526.430505,0.0,15526.430505,337.098080


In [70]:
colors = {
    'unharvestable_pods': '#ff595e', 
    'harvested_pods': "#8ac926", 
    'harvestable_pods': "#1982c4", 
    # 'unharvestable_ratio': "#6a4c93", 
} 
c = (
    chart(
        df, 
        "timestamp", 
        lmetrics=['unharvestable_pods', 'harvested_pods', 'harvestable_pods',], 
        # rmetrics=['unharvestable_ratio'],
        lstrategy="stack_area", 
        rstrategy="line",
        title="Pod Line Breakdown", 
        tooltip_formats={
            'unharvestable_pods': ',d', 
            'harvested_pods': ',d', 
            'harvestable_pods': ',d', 
            # 'unharvestable_ratio': '.0%', 
        },
        yaxis_left_kwargs=dict(title="Pods", format=".3~s"), 
        # yaxis_right_kwargs=dict(title="Ratio", format=".0%"), 
        # dual_axes=True, 
        colors=colors, 
        show_exploit_rule=True, 
        exploit_day=16, 
    )
)

css_lines = css_tooltip_timeseries_multi_colored(value_vars, colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [ ]:
# TODO: update css for this chart 
output_chart(c, css=css)

<IPython.core.display.JSON object>